In [ ]:
import pymysql
import yfinance as yf
import math
from math import sqrt
import numpy as np
import pandas as pd
import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
from pyhht.visualization import plot_imfs
from sklearn.metrics import r2_score
from datetime import datetime,timedelta
from pyhht.emd import EMD
from matplotlib import pyplot
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from matplotlib import gridspec
import pywt
from pywt import wavedec
import time

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
class MySQL:
    def __init__(self):
        super(MySQL, self).__init__()

        self.connection = pymysql.connect(
            host="",
            user="",
            password="",
            db="",
            charset='utf8',
            cursorclass=pymysql.cursors.DictCursor
        )

    def connect(self, host, user, password, db):
        self.connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            db=db,
            charset='utf8',
            cursorclass=pymysql.cursors.DictCursor
        )

    def reconnect(self, DB_CONFIG):
        self.connection = pymysql.connect(
            host=DB_CONFIG['host'],
            user=DB_CONFIG['user'],
            password=DB_CONFIG['password'],
            db=DB_CONFIG['db'],
            charset='utf8',
            cursorclass=pymysql.cursors.DictCursor
        )

    def execute(self, sql):
        connection = self.connection
        cursor = connection.cursor()
        result = cursor.execute(sql)
        connection.commit()
        response = {
            'result': result,
            'sn': cursor.lastrowid
        }

        return response

    def query(self, state):
        connection = self.connection
        with connection.cursor() as cursor:
            cursor.execute(state)
            result = cursor.fetchall()

        return result

    def ping(self):
        self.connection.ping(reconnect=True)

    def close(self):
        self.connection.close()
#整段時間區間
train_begin_date = '2019-01-01'
train_end_date = '2021-01-01'

predict_begin_date = '2021-05-01'
predict_end_date = '2021-05-31'

# sql = "SELECT * from demand order by dateㄍtime desc limit 30000"
sql = '''SELECT  `demand_quarter`, `datetime`
                FROM `demand_with_weather_data` 
                WHERE `datetime` >= '{}' AND `datetime` < '{}'
                ORDER BY `datetime` '''.format(train_begin_date, predict_end_date)
dbh = MySQL()
all_data = dbh.query(sql)
train_sql = '''SELECT  `demand_quarter`, `datetime`
                FROM `demand_with_weather_data` 
                WHERE `datetime` >= '{}' AND `datetime` < '{}'
                ORDER BY `datetime` '''.format(train_begin_date, train_end_date)
train_data = dbh.query(train_sql)
predict_sql = '''SELECT  `demand_quarter`, `datetime`
                FROM `demand_with_weather_data` 
                WHERE `datetime` >= '{}' AND `datetime` < '{}'
                ORDER BY `datetime` '''.format(predict_begin_date, predict_end_date)
predict_data = dbh.query(predict_sql)
dbh.close()
all_df = pd.DataFrame(all_data)
all_df = all_df.set_index('datetime')
train_df = pd.DataFrame(train_data)
train_df = train_df.set_index('datetime')
predict_df = pd.DataFrame(predict_data)
predict_df = predict_df.set_index('datetime')
# test_df = test_df.set_index('sn')
# predict_df
train_df

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(train_df)
# plt.plot(predict_df)

# 將已蒐集的訓練集資料先進行分解

In [ ]:
wavelet='Haar'

In [ ]:
# 訓練集的小波轉換
train = train_df['demand_quarter']
train_coefficients = pywt.wavedec(train, wavelet, level = 2)
# max_level_train = pywt.swt_max_level(len(train))
# train_coefficients = pywt.swt(train, 'db10', level=3)
len(train_coefficients[0])

In [ ]:
len(train_coefficients[0])

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(train_coefficients[0])
plt.plot(train_coefficients[1])
plt.plot(train_coefficients[2])

In [ ]:
# 測試集的小波轉換
test = predict_df['demand_quarter']
test_coefficients = pywt.wavedec(test, wavelet, level =2)
# max_level_train = pywt.swt_max_level(len(train))
# train_coefficients = pywt.swt(train, 'db10', level=3)
len(test_coefficients)

In [ ]:
plt.figure(figsize=(10,5))
# plt.xlim([600,900])
plt.plot(train_coefficients[0])

plt.plot(test_coefficients[0])

# 預訓練模型

In [ ]:
def createSlidingWindow(dataset,windowSize):
    x_train = []
    y_train = []
    for i in range(len(dataset)-windowSize):
        x_train.append(dataset[i:i+windowSize])
        y_train.append(dataset[i+windowSize])
    return np.array(x_train), np.array(y_train)

In [ ]:
batch_size = 1000
epochs = 50

In [ ]:
def LSTM_model(x, y,timestep,i,batch_size,epochs):
#     batch_size = len(x)/50
    
    scaler = MinMaxScaler(feature_range=(0,1))
    
    y_temp = np.concatenate([np.array(x[-timestep:]).flatten(), np.array(y).flatten()], axis=0)
    train_scaled_data = scaler.fit_transform(x.reshape(-1,1))
    test_scaled_data = scaler.fit_transform(y_temp.reshape(-1,1))
    
    x_train,y_train = createSlidingWindow(train_scaled_data,timestep)
    x_test,y_test = createSlidingWindow(test_scaled_data,timestep)
    print(len(y_test))
    print(len(x_test))
    #Build LSTM model
    model=Sequential()
    #Add first layer to model
    model.add(LSTM(32, return_sequences=False, input_shape=(timestep,x_train.shape[2])))
    #Add second layer to model
    # model.add(LSTM(4, return_sequences=False))
    # #Add Dense Layer to model with 25 neurons
    # model.add(Dense(4))
    #Add Dense Layer to model with 1 neuron
    # model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error',metrics = ['accuracy'],run_eagerly=True)
    model.summary()
    history = model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs)
    print(x_test)
    test_predict = model.predict(x_test)
    predict_Close = scaler.inverse_transform(test_predict)
    y_test_true = scaler.inverse_transform(y_test)
    LSTM_R2 = r2_score(y_test_true,predict_Close)
    LSTM_MSE = mean_squared_error(y_test_true,predict_Close)
    LSTM_MAE = mean_absolute_error(y_test_true,predict_Close)
    print("LSTM_R2=",LSTM_R2)
    print("LSTM_MSE=",LSTM_MSE)
    print("LSTM_MAE=",LSTM_MAE)
    print("LSTM_RMSE=",sqrt(LSTM_MSE))
    plt.figure(figsize=(10 ,5))
    plt.title("sliding_window_size:{}".format(timestep))
    plt.plot(y_test_true)
    plt.plot(predict_Close,c='g')
    plt.legend(['data', 'LSTM_prediction'], loc='upper right')
    plt.show()

    loss = history.history['loss']
    plt.plot(loss, label='loss')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    
    simple_LSTM_modal_save_path = './model/DWT_simple-LSTM-model-result-{}-{}.h5'.format(i,timestep)
    model.save(simple_LSTM_modal_save_path)  # creates a HDF5 file 'my_model.h5'
#     filters = model.layers[0].get_weights()
#     print(filters)
    
    return predict_Close

In [ ]:
LSTM_R2 = []
LSTM_MSE = []
LSTM_MAE = []
LSTM_MRE = []
LSTM_MAPE = []
for look_back in range(48,288,48):
    result_1 = LSTM_model(train_coefficients[0],test_coefficients[0],look_back,1,batch_size,epochs)
    result_2 = LSTM_model(train_coefficients[1],test_coefficients[1],look_back,2,batch_size,epochs)
    result_3 = LSTM_model(train_coefficients[2],test_coefficients[2],look_back,3,batch_size,epochs)

    final_result = []
    final_result.append(result_1.ravel())
    final_result.append(result_2.ravel())
    final_result.append(result_3.ravel())
    
    updated_original = pywt.waverec(final_result, wavelet)
    
    k = test.tolist()
    r = updated_original[:2873]
    LSTM_R2.append(r2_score(k,r))
    LSTM_MSE.append(mean_squared_error(k,r))
    LSTM_MAE.append(mean_absolute_error(k,r))
    LSTM_MRE.append(np.mean(np.abs((k - r) / k)) * 100)
    LSTM_MAPE.append((abs(r-k)/k).mean())

In [ ]:
print(LSTM_R2,LSTM_MSE,LSTM_MAE,LSTM_MRE,LSTM_MAPE)

In [ ]:
result_1 = LSTM_model(train_coefficients[0],test_coefficients[0],timestamp,1,batch_size,epochs)

In [ ]:
result_2 = LSTM_model(train_coefficients[1],test_coefficients[1],timestamp,2,batch_size,epochs)

In [ ]:
result_3 = LSTM_model(train_coefficients[2],test_coefficients[2],timestamp,3,batch_size,epochs)

In [ ]:
result_3_zero = []
for i in range(len(result_3)):
    result_3_zero.append([0])
result_3_zero = np.array(result_3_zero)
result_3_zero

result_2_zero = []
for i in range(len(result_2)):
    result_2_zero.append([0])
result_2_zero = np.array(result_2_zero)
result_2_zero

In [ ]:
final_result = []
final_result.append(result_1.ravel())
final_result.append(result_2.ravel())
final_result.append(result_3_zero.ravel())
final_result

In [ ]:
updated_original = pywt.waverec(final_result, wavelet)

In [ ]:
plt.figure(figsize=(10 ,5))

k = test.tolist()
r = updated_original[:2873]

LSTM_R2 = r2_score(test.tolist(),updated_original[:2873])
LSTM_MSE = mean_squared_error(test.tolist(),updated_original[:2873])
LSTM_MAE = mean_absolute_error(test.tolist(),updated_original[:2873])
LSTM_MRE = np.mean(np.abs((k - r) / k)) * 100

plt.xlim([200,500])
plt.ylabel("demand")
plt.xlabel("minutes")
print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))
print("LSTM_MRE=",LSTM_MRE)

plt.plot(test.tolist())
plt.plot(updated_original,c='orange')
plt.legend(['data', 'DWT_LSTM_prediction'], loc='upper right')
plt.show()

# 抓新資料少量分解更新子訊號

In [ ]:
# 取一段時間的電力資料，傳給DWT_decompose()
def take_latest_several_data(end_date):

    conn  =  pymysql.connect ( host = '' ,  user = '' ,  passwd = "" ,  db = '' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `datetime`, `demand_quarter`
                    FROM `demand_with_weather_data`
                    WHERE `datetime` < '{}'
                    ORDER BY `datetime` DESC LIMIT {} '''.format(end_date,96)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]

    reframed = pd.DataFrame(results_values_list)
    #         print(reframed)
    reframed.columns = result_key_list
    timestamp = pd.to_datetime(reframed.datetime, infer_datetime_format=True).values
    reframed['timestamp'] = timestamp.tolist()
    reframed = reframed.set_index('datetime')
    cur.close () 
    conn.close()
    reframed['demand_quarter'] = reframed['demand_quarter'].replace('', np.nan)
    reframed['demand_quarter'] = reframed['demand_quarter'].astype(float) 
    electricity_new_data_df = reframed
    electricity_new_data_df = electricity_new_data_df.drop('timestamp', axis=1)
    electricity_new_data = electricity_new_data_df['demand_quarter'].to_list()
    return  electricity_new_data_df[:96],list(reversed(electricity_new_data[:96]))

In [ ]:
a,b = take_latest_several_data('2021-01-02 00:45:00')

In [ ]:
def DWT_decompose(coeff_num,old_data_list, end_date, wavelet, level):
    financial_new_data_df,financial_new_data = take_latest_several_data(end_date)
    print(financial_new_data)
    new_data_coeff = pywt.wavedec(financial_new_data, wavelet, level=level)
    print(new_data_coeff)
    
    if coeff_num == 1:
        old_data_list[0] = np.append(old_data_list[0],new_data_coeff[0])
    elif coeff_num == 2:
        old_data_list[1] = np.append(old_data_list[1],new_data_coeff[1])
    elif coeff_num == 3:
        old_data_list[2] = np.append(old_data_list[2],new_data_coeff[2])
#     print(new_data_coeff)
    return old_data_list,new_data_coeff

# 增量預測

In [ ]:
def predict_multistep(model, input_data, n_steps_out):
    predictions = []
    current_input = input_data
#     scaler = MinMaxScaler(feature_range=(0,1))
    
    for _ in range(n_steps_out):
        prediction = model.predict(current_input)
#         print('*******************',prediction)
#         predict_Close = scaler.inverse_transform(prediction[0][0])
        predictions.append(prediction)
#         print('++++++++++++++++++++',current_input)
        current_input = np.append(current_input[0:, 0:], np.expand_dims(prediction, axis=1), axis=1)
#         print('--------------------',current_input)
    return np.array(predictions)
#
# predict multistep ver.1

In [ ]:
def predict_electricity(coeff,coeff_num, model_path,old_data_list,date_range_list,wavelet):

    window_size = 96
    feature_len = 1
    
    scaler = MinMaxScaler(feature_range=(0,1))
    test = np.array(coeff[-(window_size+1):])
    test = test[::-1]
#     print(test)
#     train_scaled_data = scaler.fit_transform(np.array(reframed_supervised['Close_^TWII(t)']).reshape(-1,1))
    test_scaled_data = scaler.fit_transform(test.reshape(-1,1))
#     print(len(test_scaled_data))
    x_test,y_test = createSlidingWindow(test_scaled_data,window_size)
    vanilla_model = load_model(model_path)
    final_predict = []
    if coeff_num == 3: 
        even_test_scaled_data = []
        odd_test_scaled_data = []
        even_test_scaled_data = test_scaled_data[::2]
        
#         print(even_test_scaled_data)
        test_predict = vanilla_model.predict(test_scaled_data)
        predict_Close = scaler.inverse_transform(test_predict)
        final_predict.append(predict_Close[0][0])

#         print(coeff)
        multi_predict = coeff
        multi_predict = np.append(multi_predict,predict_Close[0][0])
#         print('-------------------',multi_predict[-10:])
#         for i in range(1,int(len(test_scaled_data)/2)+1,2):
        odd_test_scaled_data = test_scaled_data[1::2]
        
        test = np.array(multi_predict[-(window_size+1):])
        test = test[::-1]
        test_scaled_data = scaler.fit_transform(test.reshape(-1,1))
#         x_test,y_test = createSlidingWindow(test_scaled_data,window_size)
        test_predict = vanilla_model.predict(test_scaled_data)
        predict_Close = scaler.inverse_transform(test_predict)
        final_predict.append(predict_Close[0][0])
    else:
#         print('原始: ',x_test)
        test_predict = vanilla_model.predict(test_scaled_data)
#         print("預測: ",test_predict)
        predict_Close = scaler.inverse_transform(test_predict)
        final_predict.append(predict_Close[0][0])

    data_list,new_data_coeff = DWT_decompose(coeff_num,old_data_list,date_range_list,wavelet,2)
#     print(data_list[2][-10:])
    test_new = np.array(data_list[coeff_num-1])
    test_new = test_new[::-1]
#     print('--------',len(test_new),test_new)
    test_scaled_data_new = scaler.fit_transform(test_new.reshape(-1,1))
#     print('--------',len(test_scaled_data_new[:window_size,:]),test_scaled_data_new[:window_size,:])
    x_test_new,y_test_new = createSlidingWindow(test_scaled_data_new[:(window_size+1),:],window_size)
    vanilla_model.compile(loss='mse', optimizer='Adam') 
#     print('-----',y_test_new)
    vanilla_model_history = vanilla_model.fit(test_scaled_data_new[1:(window_size+1),:],test_scaled_data_new[:window_size,:],batch_size=60,epochs=100)
#         all_update_features, update_labels)

    vanilla_model.save(model_path) 
    del vanilla_model
    print('+++',final_predict)
    return final_predict

In [ ]:
def predict_date_range(old_data_list,date_range_list,wavelet):
    predict_list = []
    execute_time_list = []
    all_time_start = time.time()
    print(date_range_list)
#     for i in range(len(date_range_list)):
    print(i,date_range_list)
    start = time.time()
#     data_list,new_data_coeff = DWT_decompose(old_data_list,date_range_list,wavelet,2)


#     new_data_coeff = pywt.wavedec(old_data_list, wavelet, level=2)
    coefficient1_result = predict_electricity(old_data_list[0],1,'./model/DWT_simple-LSTM-model-result-1-Copy1.h5',old_data_list,date_range_list,wavelet)
    coefficient2_result = predict_electricity(old_data_list[1],2,'./model/DWT_simple-LSTM-model-result-2-Copy1.h5',old_data_list,date_range_list,wavelet)
    coefficient3_result = predict_electricity(old_data_list[2],3,'./model/DWT_simple-LSTM-model-result-3-Copy1.h5',old_data_list,date_range_list,wavelet)
    

#     print(coefficient1_result)
    
    predict_coeff = []
    temp = []
    predict_coeff.append(np.array(coefficient1_result))
    predict_coeff.append(np.array(coefficient2_result))
#     temp.append(np.array(coefficient3_result).ravel())
#     temp.append(np.array(coefficient3_result).ravel())
    predict_coeff.append(np.array(coefficient3_result))
#     predict_coeff.append(coefficient4_result[0].ravel())
#     predict_coeff.append(np.array(temp).ravel())
#     predict_coeff.append(np.array(0).ravel())
#     predict_coeff.append(np.array(0).ravel())
#     print(predict_coeff)
    
    predict_result = pywt.waverec(predict_coeff, wavelet)
    predict_list.append(predict_result)

    
    end = time.time()
    execute_time_list.append(end - start)
    all_time_end = time.time()
    print("執行時間：%f 秒" % (all_time_end - all_time_start))
    return predict_list,coefficient1_result,coefficient2_result,coefficient3_result

In [ ]:
date_range_list_1 = pd.date_range(start='2021-05-01 00:00:00', end='2021-05-03 23:45:00', freq='15min')

In [ ]:
date_range_list_2 = pd.date_range(start='2021-05-04 00:00:00', end='2021-05-06 23:45:00', freq='15min')

In [ ]:
date_range_list_3 = pd.date_range(start='2021-05-07 00:00:00', end='2021-05-09 23:45:00', freq='15min')

In [ ]:
date_range_list_test384 = pd.date_range(start='2021-05-01 00:00:00', end='2021-05-15 23:45:00', freq='15min')

In [ ]:
date_range_list_1

In [ ]:
len(date_range_list_test384)

In [ ]:
train_coefficient_append = train_coefficients
decompose_start = time.time()
for i in range(int(len(date_range_list_test384)/4)):
#     print(date_range_list_1[i])
    data_list,new_data_coeff = DWT_decompose(1,train_coefficient_append,date_range_list_test384[4*i+3],wavelet,2)
decompose_end = time.time()
decompose_time = decompose_end - decompose_start
print("分解時間:%f 秒" % decompose_time)

In [ ]:
train_coefficient_append = train_coefficients
decompose_start = time.time()
for i in range(int(len(date_range_list_test384)/384)):
#     print(date_range_list_1[i])
    data_list,new_data_coeff = DWT_decompose(1,train_coefficient_append,date_range_list_test384[384*i+3],wavelet,2)
decompose_end = time.time()
decompose_time = decompose_end - decompose_start
print("分解時間:%f 秒" % decompose_time)

In [ ]:
predict_list_1 = []
coeff_predict_list_1 = []
coeff_1_predict_list_1 = []
coeff_2_predict_list_1 = []
coeff_3_predict_list_1 = []
all_time_start = time.time()
# train_coefficient_append = train_coefficients
for i in range(int(len(date_range_list_1)/4)):
#     print(date_range_list_1[i])
    predict_value,coeff1,coeff2,coeff3 = predict_date_range(train_coefficients,date_range_list_1[4*i+3],wavelet)
    predict_list_1.append(predict_value)
    coeff_1_predict_list_1.append(coeff1)
    coeff_2_predict_list_1.append(coeff2)
    coeff_3_predict_list_1.append(coeff3)
#     print(predict_list)

#     print('--------------',train_coefficients[2][-30:])
#     renew_model(1,'./model/DWT_simple-LSTM-model-result-1-Copy1.h5',train_coefficients,date_range_list_1[4*i],wavelet)
#     renew_model(2,'./model/DWT_simple-LSTM-model-result-2-Copy1.h5',train_coefficients,date_range_list_1[4*i],wavelet)
#     renew_model(3,'./model/DWT_simple-LSTM-model-result-3-Copy1.h5',train_coefficients,date_range_list_1[4*i],wavelet)
coeff_predict_list_1.append(coeff_1_predict_list_1)
coeff_predict_list_1.append(coeff_2_predict_list_1)
coeff_predict_list_1.append(coeff_3_predict_list_1)
all_time_end = time.time()
print("全部執行時間：%f 秒" % (all_time_end - all_time_start))

In [ ]:
predict_list_s1 = []
coeff_predict_list_s1 = []
coeff_1_predict_list_s1 = []
coeff_2_predict_list_s1 = []
coeff_3_predict_list_s1 = []
all_time_start = time.time()
# train_coefficient_append = train_coefficients
for i in range(int(len(date_range_list_1)/4)):
#     print(date_range_list_1[i])
    predict_value,coeff1,coeff2,coeff3 = predict_date_range(train_coefficients,date_range_list_1[4*i+3],wavelet)
    predict_list_s1.append(predict_value)
    coeff_1_predict_list_s1.append(coeff1)
    coeff_2_predict_list_s1.append(coeff2)
    coeff_3_predict_list_s1.append(coeff3)
#     print(predict_list)

#     print('--------------',train_coefficients[2][-30:])
#     renew_model(1,'./model/DWT_simple-LSTM-model-result-1-Copy1.h5',train_coefficients,date_range_list_1[4*i],wavelet)
#     renew_model(2,'./model/DWT_simple-LSTM-model-result-2-Copy1.h5',train_coefficients,date_range_list_1[4*i],wavelet)
#     renew_model(3,'./model/DWT_simple-LSTM-model-result-3-Copy1.h5',train_coefficients,date_range_list_1[4*i],wavelet)
coeff_predict_list_s1.append(coeff_1_predict_list_s1)
coeff_predict_list_s1.append(coeff_2_predict_list_s1)
coeff_predict_list_s1.append(coeff_3_predict_list_s1)
all_time_end = time.time()
print("全部執行時間：%f 秒" % (all_time_end - all_time_start))

In [ ]:
predict_list_2 = []
coeff_predict_list_2 = []
coeff_1_predict_list_2 = []
coeff_2_predict_list_2 = []
coeff_3_predict_list_2 = []
all_time_start = time.time()
# train_coefficient_append = train_coefficients
for i in range(int(len(date_range_list_2)/4)):
#     print(date_range_list_1[i])
    predict_value,coeff1,coeff2,coeff3 = predict_date_range(train_coefficients,date_range_list_2[4*i+3],wavelet)
    predict_list_2.append(predict_value)
    coeff_1_predict_list_2.append(coeff1)
    coeff_2_predict_list_2.append(coeff2)
    coeff_3_predict_list_2.append(coeff3)
#     print(predict_list)

#     print('--------------',train_coefficients[2][-30:])
#     renew_model(1,'./model/DWT_simple-LSTM-model-result-1-Copy1.h5',train_coefficients,date_range_list_1[4*i],wavelet)
#     renew_model(2,'./model/DWT_simple-LSTM-model-result-2-Copy1.h5',train_coefficients,date_range_list_1[4*i],wavelet)
#     renew_model(3,'./model/DWT_simple-LSTM-model-result-3-Copy1.h5',train_coefficients,date_range_list_1[4*i],wavelet)
coeff_predict_list_2.append(coeff_1_predict_list_2)
coeff_predict_list_2.append(coeff_2_predict_list_2)
coeff_predict_list_2.append(coeff_3_predict_list_2)
all_time_end = time.time()
print("全部執行時間：%f 秒" % (all_time_end - all_time_start))

In [ ]:
predict_list_3 = []
coeff_predict_list_3 = []
coeff_1_predict_list_3 = []
coeff_2_predict_list_3 = []
coeff_3_predict_list_3 = []
all_time_start = time.time()
# train_coefficient_append = train_coefficients
for i in range(int(len(date_range_list_3)/4)):
#     print(date_range_list_1[i])
    predict_value,coeff1,coeff2,coeff3 = predict_date_range(train_coefficients,date_range_list_3[4*i+3],wavelet)
    predict_list_3.append(predict_value)
    coeff_1_predict_list_3.append(coeff1)
    coeff_2_predict_list_3.append(coeff2)
    coeff_3_predict_list_3.append(coeff3)
#     print(predict_list)

#     print('--------------',train_coefficients[2][-30:])
#     renew_model(1,'./model/DWT_simple-LSTM-model-result-1-Copy1.h5',train_coefficients,date_range_list_1[4*i],wavelet)
#     renew_model(2,'./model/DWT_simple-LSTM-model-result-2-Copy1.h5',train_coefficients,date_range_list_1[4*i],wavelet)
#     renew_model(3,'./model/DWT_simple-LSTM-model-result-3-Copy1.h5',train_coefficients,date_range_list_1[4*i],wavelet)
coeff_predict_list_3.append(coeff_1_predict_list_3)
coeff_predict_list_3.append(coeff_2_predict_list_3)
coeff_predict_list_3.append(coeff_3_predict_list_3)
all_time_end = time.time()
print("全部執行時間：%f 秒" % (all_time_end - all_time_start))

In [ ]:
predict_list_1 = np.array(predict_list_1).ravel()

In [ ]:
predict_list_s1 = np.array(predict_list_s1).ravel()

In [ ]:
predict_list_2 = np.array(predict_list_2).ravel()

In [ ]:
predict_list_3 = np.array(predict_list_3).ravel()

In [ ]:
len(predict_list_1)

# 比較實際值與預測結果

In [ ]:
# 取真實電力資料的dataframe
def get_true_value(start_day,end_day):
    conn  =  pymysql.connect ( host = '' ,  user = '' ,  passwd = "" ,  db = '' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `datetime`, `demand_quarter`
                    FROM `demand_with_weather_data`
                    WHERE `datetime` >= '{}' AND `datetime` <= '{}' '''.format(start_day, end_day)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    true_day_value = pd.DataFrame(results_values_list)
    true_day_value.columns = result_key_list
    true_day_value = true_day_value.set_index('datetime')
    cur.close () 
    conn.close()

    true_day_value['demand_quarter'] = true_day_value['demand_quarter'].replace('', np.nan)
    true_day_value['demand_quarter'] = true_day_value['demand_quarter'].astype(float)  
    return true_day_value
    # datelist = true_day_value.index

In [ ]:
# 將真實資料與預測值合併成一個dataframe
def concat_true_and_predict_value(start_day, end_day,date_range_list, predict_list):
    true_day_value = get_true_value(start_day,end_day)
#     predict_datelist = datelist(start_day,end_day)
    predict_datelist = date_range_list
    predict_df = pd.DataFrame({'datetime': predict_datelist[:960], 'predict': predict_list})
#     execute_time_df = pd.DataFrame({'datetime': predict_datelist, 'execute_time': execute_list})
    predict_df = predict_df.set_index('datetime')
#     execute_time_df = execute_time_df.set_index('datetime')
    merged_df = pd.concat([true_day_value, predict_df], axis=1)
#     merged_df = pd.concat([merged_df, execute_time_df], axis=1)
    merged_df = merged_df.dropna()
    return merged_df

In [ ]:
len(date_range_list_1)

In [ ]:
len(predict_list_1)

In [ ]:
concat_df_1 = concat_true_and_predict_value('2021-05-01 00:00:00','2021-05-03 23:45:00',date_range_list_1,predict_list_1)
concat_df_1

In [ ]:
concat_df_s1 = concat_true_and_predict_value('2021-05-01 00:00:00','2021-05-03 23:45:00',date_range_list_1,predict_list_s1)
concat_df_s1

In [ ]:
# sliding window denoise
result_s3_zero = []
for i in range(2*len(coeff_3_predict_list_s1)):
    result_s3_zero.append([0])
result_s3_zero = np.array(result_s3_zero)
# print(len(coeff_3_predict_list_1))

result_s2_zero = []
for i in range(len(coeff_2_predict_list_s1)):
    result_s2_zero.append([0])
result_s2_zero = np.array(result_s2_zero)
# print(len(coeff_2_predict_list_1))

thr = 10.3654

thress2 = pywt.threshold(coeff_2_predict_list_s1, thr, 'hard')
thress3 = pywt.threshold(coeff_3_predict_list_s1, thr, 'hard')

final_result_s = []
final_result_s.append(np.array(coeff_1_predict_list_s1).ravel())
final_result_s.append(np.array(thress2).ravel())
final_result_s.append(np.array(thress3).ravel())
# print(final_result)
updated_original_s = pywt.waverec(final_result_s, wavelet)

In [ ]:
result_3_zero = []
for i in range(2*len(coeff_3_predict_list_1)):
    result_3_zero.append([0])
result_3_zero = np.array(result_3_zero)
# print(len(coeff_3_predict_list_1))

result_2_zero = []
for i in range(len(coeff_2_predict_list_1)):
    result_2_zero.append([0])
result_2_zero = np.array(result_2_zero)
# print(len(coeff_2_predict_list_1))

thr = 10.3654

thres2 = pywt.threshold(coeff_2_predict_list_1, thr, 'hard')
thres3 = pywt.threshold(coeff_3_predict_list_1, thr, 'hard')

final_result = []
final_result.append(np.array(coeff_1_predict_list_1).ravel())
final_result.append(np.array(thres2).ravel())
final_result.append(np.array(thres3).ravel())
# print(final_result)
updated_original = pywt.waverec(final_result, wavelet)

plt.figure(figsize=(10 ,5))
# plt.ylim([13500,20000])
# plt.xlim([0,100])
# plt.ylim([12000,19000])
plt.plot(concat_df_1['demand_quarter'].to_list())
plt.plot(updated_original_s,c='violet')
plt.plot(updated_original,c='orange')

plt.legend(['data', 'DWT_prediction','DWT_prediction(incremental)'], loc='upper right')
plt.ylim([100,350])
plt.show()

k = concat_df_1['demand_quarter']
r = updated_original_s

LSTM_R2 = r2_score(concat_df_1['demand_quarter'],updated_original_s)
LSTM_MSE = mean_squared_error(concat_df_1['demand_quarter'],updated_original_s)
LSTM_MAE = mean_absolute_error(concat_df_1['demand_quarter'],updated_original_s)
LSTM_MRE = np.mean(np.abs((k - r) / k)) * 100
print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))
print("LSTM_MRE=",LSTM_MRE)

In [ ]:
coeff_2_predict_list_1

In [ ]:
h = pd.concat([concat_df_1, pd.DataFrame(updated_original)])
h

In [ ]:
plt.figure(figsize=(10 ,5))
# plt.ylim([13500,20000])464M0-LK83Q-0Y9B8
# plt.xlim([0,100])
# plt.ylim([12000,19000])
plt.plot(concat_df_1['demand_quarter'].to_list())
plt.plot(concat_df_1['predict'].to_list(),c='orange')
# plt.xlim([100,300])
plt.show()

LSTM_R2 = r2_score(concat_df_1['demand_quarter'],concat_df_1['predict'])
LSTM_MSE = mean_squared_error(concat_df_1['demand_quarter'],concat_df_1['predict'])
LSTM_MAE = mean_absolute_error(concat_df_1['demand_quarter'],concat_df_1['predict'])
print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))

In [ ]:
concat_df_2 = concat_true_and_predict_value('2021-05-04 00:00:00','2021-05-06 23:45:00',date_range_list_2,predict_list_2)
concat_df_2

In [ ]:
result_3_zero = []
for i in range(2*len(coeff_3_predict_list_2)):
    result_3_zero.append([0])
result_3_zero = np.array(result_3_zero)
# print(len(coeff_3_predict_list_1))

result_2_zero = []
for i in range(len(coeff_2_predict_list_2)):
    result_2_zero.append([0])
result_2_zero = np.array(result_2_zero)
# print(len(coeff_2_predict_list_1))


thr = 3.3654
thres2 = pywt.threshold(coeff_2_predict_list_2, thr, 'hard')
thres3 = pywt.threshold(coeff_3_predict_list_2, thr, 'hard')

final_result = []
final_result.append(np.array(coeff_1_predict_list_2).ravel())
final_result.append(np.array(thres2).ravel())
final_result.append(np.array(thres3).ravel())
# print(final_result)
updated_original = pywt.waverec(final_result, wavelet)

plt.figure(figsize=(10 ,5))
# plt.ylim([13500,20000])
# plt.xlim([0,100])
# plt.ylim([12000,19000])
plt.plot(concat_df_2['demand_quarter'].to_list())
plt.plot(updated_original,c='orange')
plt.legend(['data', 'DWT_LSTM_prediction'], loc='upper right')
plt.ylim([100,500])
# plt.xlim([100,150])
plt.show()

LSTM_R2 = r2_score(concat_df_2['demand_quarter'],updated_original)
LSTM_MSE = mean_squared_error(concat_df_2['demand_quarter'],updated_original)
LSTM_MAE = mean_absolute_error(concat_df_2['demand_quarter'],updated_original)
k = concat_df_2['demand_quarter']
r = updated_original
LSTM_MRE = np.mean(np.abs((k - r) / k)) * 100
print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))
print("LSTM_MRE=",LSTM_MRE)

In [ ]:
pd.DataFrame(updated_original)

In [ ]:
plt.figure(figsize=(10 ,5))
# plt.ylim([13500,20000])464M0-LK83Q-0Y9B8
# plt.xlim([1,200])
# plt.ylim([12000,19000])
plt.plot(concat_df_2['demand_quarter'].to_list())
plt.plot(concat_df_2['predict'].to_list(),c='orange')
# plt.xlim([100,300])
plt.show()

LSTM_R2 = r2_score(concat_df_2['demand_quarter'],concat_df_2['predict'])
LSTM_MSE = mean_squared_error(concat_df_2['demand_quarter'],concat_df_2['predict'])
LSTM_MAE = mean_absolute_error(concat_df_2['demand_quarter'],concat_df_2['predict'])
print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))

In [ ]:
concat_df_3 = concat_true_and_predict_value('2021-05-07 00:00:00','2021-05-09 23:45:00',date_range_list_3,predict_list_3)
concat_df_3

In [ ]:
result_3_zero = []
for i in range(2*len(coeff_3_predict_list_3)):
    result_3_zero.append([0])
result_3_zero = np.array(result_3_zero)
# print(len(coeff_3_predict_list_1))

result_2_zero = []
for i in range(len(coeff_2_predict_list_3)):
    result_2_zero.append([0])
result_2_zero = np.array(result_2_zero)
# print(len(coeff_2_predict_list_1))



thr = 5.3654
thres2 = pywt.threshold(coeff_2_predict_list_3, thr, 'hard')
thres3 = pywt.threshold(coeff_3_predict_list_3, thr, 'hard')

final_result = []
final_result.append(np.array(coeff_1_predict_list_3).ravel())
final_result.append(np.array(thres2).ravel())
final_result.append(np.array(thres3).ravel())
# print(final_result)
updated_original = pywt.waverec(final_result, wavelet)

plt.figure(figsize=(10 ,5))

# plt.ylim([100,350])
plt.plot(concat_df_3['demand_quarter'].to_list())
plt.plot(updated_original,c='orange')
plt.legend(['data', 'DWT_LSTM_prediction'], loc='upper right')
# plt.xlim([100,300])
plt.show()

LSTM_R2 = r2_score(concat_df_3['demand_quarter'],updated_original)
LSTM_MSE = mean_squared_error(concat_df_3['demand_quarter'],updated_original)
LSTM_MAE = mean_absolute_error(concat_df_3['demand_quarter'],updated_original)
k = concat_df_3['demand_quarter']
r = updated_original
LSTM_MRE = np.mean(np.abs((k - r) / k)) * 100
print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))
print("LSTM_MRE=",LSTM_MRE)

In [ ]:
coeff_2_predict_list_3

In [ ]:
plt.figure(figsize=(10 ,5))
# plt.ylim([13500,20000])464M0-LK83Q-0Y9B8
# plt.xlim([1,200])
# plt.ylim([12000,19000])
plt.plot(concat_df_3['demand_quarter'].to_list())
plt.plot(concat_df_3['predict'].to_list(),c='orange')
# plt.xlim([0,100])
plt.show()

LSTM_R2 = r2_score(concat_df_3['demand_quarter'],concat_df_3['predict'])
LSTM_MSE = mean_squared_error(concat_df_3['demand_quarter'],concat_df_3['predict'])
LSTM_MAE = mean_absolute_error(concat_df_3['demand_quarter'],concat_df_3['predict'])
print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))

# 訊號重建觀察

In [ ]:
plt.figure(figsize=(10 ,5))
# plt.ylim([13500,20000])464M0-LK83Q-0Y9B8
# plt.xlim([1,200])
# plt.ylim([12000,19000])
plt.plot(concat_df_1['demand_quarter'].to_list())
plt.plot(concat_df_1['predict'].to_list(),c='g')
# plt.xlim([100,300])
plt.show()

LSTM_R2 = r2_score(concat_df_1['demand_quarter'],concat_df_1['predict'])
LSTM_MSE = mean_squared_error(concat_df_1['demand_quarter'],concat_df_1['predict'])
LSTM_MAE = mean_absolute_error(concat_df_1['demand_quarter'],concat_df_1['predict'])
print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))

In [ ]:
coeff_reconstruct = pywt.waverec(train_coefficients, wavelet)

In [ ]:
len(coeff_reconstruct)

In [ ]:
train

In [ ]:
train_begin_t = '2021-05-01 00:00:00'
train_end_t = '2021-05-04 23:45:00'


conn  =  pymysql.connect ( host = '' ,  user = '' ,  passwd = "" ,  db = '' ) 
cur  =  conn.cursor() 
dbh = MySQL()
train_sql = '''SELECT  `demand_quarter`, `datetime`
                FROM `demand_with_weather_data` 
                WHERE `datetime` >= '{}' AND `datetime` <= '{}'
                ORDER BY `datetime` '''.format(train_begin_t, train_end_t)
train_and_predict_data = dbh.query(train_sql)
dbh.close()
train_and_predict_df = pd.DataFrame(train_and_predict_data)
train_and_predict_df = train_and_predict_df.set_index('datetime')


In [ ]:
len(train_and_predict_df)

In [ ]:
len(train_df)

In [ ]:
# 訓練集的小波轉換
train_rec = train_and_predict_df['demand_quarter']
train_coefficientrec = pywt.wavedec(train_rec, wavelet, level = 2)
# max_level_train = pywt.swt_max_level(len(train))
# train_coefficients = pywt.swt(train, 'db10', level=3)
len(train_coefficientrec[0])

In [ ]:
len(train_coefficients[0])

In [ ]:
test_coeff_predict_list = pywt.wavedec(concat_df_1['predict'].to_list(), 'db1')
test_coeff_predict_list

In [ ]:
test_coeff_predict_list
test_coeff_predict_list[-1] = np.zeros_like(test_coeff_predict_list[-1])
test_coeff_predict_list[-2] = np.zeros_like(test_coeff_predict_list[-2])

filtered_data_dwt=pywt.waverec(test_coeff_predict_list,'db1',mode='symmetric',axis=-1)

In [ ]:
filtered_data_dwt

In [ ]:
import matplotlib.pyplot as plt

from skimage.restoration import (denoise_wavelet, estimate_sigma)
from skimage import data, img_as_float
from skimage.util import random_noise
from skimage.metrics import peak_signal_noise_ratio
sigma = 2
# data_denoise = denoise_wavelet(np.array(decompose_data_1), multichannel=True, convert2ycbcr=True, method='BayesShrink', mode='soft', wavelet_levels=2, wavelet=wavelet)
data_denoise = denoise_wavelet(np.array(coeff_predict_list[0]).ravel(), sigma=sigma, wavelet=wavelet, wavelet_levels=2)

plt.figure(figsize=(10 ,5))
# plt.xlim([0,20])
plt.plot(train_coefficientrec[0])
plt.plot(data_denoise,c='r')

In [ ]:
data_denoise_list = []
data_denoise_list.append(data_denoise)
data_denoise1 = denoise_wavelet(np.array(coeff_predict_list[1]).ravel(), sigma=sigma, wavelet=wavelet, wavelet_levels=2)
data_denoise_list.append(data_denoise1)
data_denoise2 = denoise_wavelet(np.array(coeff_predict_list[2]).ravel(), sigma=sigma, wavelet=wavelet, wavelet_levels=2)
data_denoise_list.append(data_denoise2)

# print(data_denoise_list)
data_denoise_rec = pywt.waverec(data_denoise_list, wavelet)

In [ ]:
plt.figure(figsize=(10 ,5))
# plt.ylim([13500,20000])464M0-LK83Q-0Y9B8
# plt.xlim([1,200])
# plt.ylim([12000,19000])
plt.plot(concat_df_1['demand_quarter'].to_list())
plt.plot(data_denoise_rec,c='r')
# plt.xlim([100,300])
plt.show()

LSTM_R2 = r2_score(concat_df_1['demand_quarter'],concat_df_1['predict'])
LSTM_MSE = mean_squared_error(concat_df_1['demand_quarter'],concat_df_1['predict'])
LSTM_MAE = mean_absolute_error(concat_df_1['demand_quarter'],concat_df_1['predict'])
print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))

In [ ]:
plt.figure(figsize=(10 ,5))
plt.plot(train_coefficients[0])
plt.plot(train_coefficientrec[0].tolist(),c='g')

In [ ]:
plt.figure(figsize=(10 ,5))
plt.plot(train_coefficients[1][-240:])
plt.plot(train_coefficientrec[1].tolist(),c='g')

In [ ]:
plt.figure(figsize=(10 ,5))
# plt.plot(train_coefficients[2][1220:])
plt.plot(train_coefficients[2][-144:])
plt.plot(train_coefficientrec[2].tolist(),c='g')



In [ ]:
plt.figure(figsize=(10 ,5))
plt.plot(coeff_reconstruct[-288:])
plt.plot(train_and_predict_df['demand_quarter'].tolist(),c='g')

In [ ]:
len(coeff_reconstruct)

In [ ]:
len(train_and_predict_df)